In [9]:
import os
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

#from tensorflow import *   # Tensorflow still doesnt seem to integrate with mac 
                            # even after installing the libraries it says 
                            # "zsh: illegal hardware instruction  /usr/local/bin/python3"
#from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
data = []
labels = []

for color in ['blue_0', 'red_0','yellow_0','green_0']:
    folder_path = f"myDataset/{color}"
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            hist = cv2.calcHist([img], [1, 2], None, [8, 8], [0, 256, 0, 256])
            
            hist = cv2.normalize(hist, hist).flatten()
            data.append(hist)
            labels.append(color)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)

# Evaluate the model on the testing data
y_pred = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Test the model on live camera feed
vc = cv2.VideoCapture(0) 
while vc.isOpened():
    rval, frame = vc.read()    # read video frames again at each loop, as long as the stream is open
    
    # Predict the color and display it on the live feed
    img = cv2.resize(frame, (224, 224))
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    hist = cv2.calcHist([img_lab], [1, 2], None, [8, 8], [0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    prediction = knn_model.predict([hist])[0]
    cv2.putText(frame, prediction, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow("stream", frame)
    
    key = cv2.waitKey(1)       # allows user intervention without stopping the stream (pause in ms)
    if key == 27:              # exit on ESC
        break
cv2.destroyWindow("stream")    # close image window upon exit
vc.release()






#CNN TENSORFLOW
# Define the data directory and image dimensions
data_dir = 'myDataset'
img_width, img_height = 64, 64

# Create an instance of ImageDataGenerator for data preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=10,  # Randomly rotate images by up to 10 degrees
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=True,  # Randomly flip images vertically
    validation_split=0.2  # Split dataset into training and validation subsets
)

# Generate training data from the dataset folder
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Generate validation data from the dataset folder
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)




# Generate training data from the dataset folder
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Generate validation data from the dataset folder
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


# Create a sequential model
model = Sequential()

# Add layers to the model
# Add a convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))

# Add a max pooling layer with pool size of 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer with 64 filters, a 3x3 kernel, and 'relu' activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another max pooling layer with pool size of 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the previous layer
model.add(Flatten())

# Add a fully connected (dense) layer with 128 units and 'relu' activation
model.add(Dense(128, activation='relu'))

# Add the output layer with appropriate number of units and activation function
model.add(Dense(num_classes, activation='softmax'))
# ... define your model architecture ...

# Add input layer with 64 units and 'relu' activation
model.add(Dense(64, activation='relu', input_dim=784))

# Add a hidden layer with 128 units and 'relu' activation
model.add(Dense(128, activation='relu'))

# Add an output layer with 10 units for multi-class classification and 'softmax' activation
model.add(Dense(10, activation='softmax'))

# Define input layer
#inputs = Input(shape=(784,))

# Add a hidden layer with 64 units and 'relu' activation
x = Dense(64, activation='relu')(inputs)

# Add another hidden layer with 128 units and 'relu' activation
x = Dense(128, activation='relu')(x)

# Define output layer with 10 units for multi-class classification and 'softmax' activation
outputs = Dense(10, activation='softmax')(x)

# Create a functional model
model = Model(inputs=inputs, outputs=outputs)


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the ImageDataGenerator
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


ModuleNotFoundError: No module named 'tensorflow'

In [10]:
pip3.11 install --upgrade pip

SyntaxError: invalid syntax (121798727.py, line 1)